In [1]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
pd.set_option('display.max_rows', None)  # Sin límite de filas

In [2]:
# cargamos el dataframe creado:
df_perros = pd.read_excel("C:/Users/isaw9/OneDrive/Escritorio/perros.xlsx", index_col = None)
df_perros

,Raza,Pelo,Peso_medio,Comida_fav
0,Chihuahua,corto,3,salmon
1,Galgo,corto,17 kg,pollo
2,Dalmata,corto,NaN,pollo
3,Boxer,corto,15.5,NaN
4,Braco,corto,15,beef
5,Doberman,NaN,30,beef
6,Pequines,largo,4,salmon
7,Bobtail,largo,NaN,pollo
8,Pastor,largo,25,pollo
9,Terrier,corto,6,pavo


In [ ]:
# Transformaciones y limpieza:

# Raza --> OK
# Pelo --> Formato OK, presencia NaN
# Peso_medio --> floats, int, NaN, kg en la str...
# Comida_fav --> formato ok, presencia NaN

In [4]:
df_perros['Peso_medio'] =pd.to_numeric( df_perros['Peso_medio'].astype(str).str.replace("kg", "", regex=False), errors ='coerce')

In [5]:
display(df_perros)

,Raza,Pelo,Peso_medio,Comida_fav
0,Chihuahua,corto,3.0,salmon
1,Galgo,corto,17.0,pollo
2,Dalmata,corto,NaN,pollo
3,Boxer,corto,15.5,NaN
4,Braco,corto,15.0,beef
5,Doberman,NaN,30.0,beef
6,Pequines,largo,4.0,salmon
7,Bobtail,largo,NaN,pollo
8,Pastor,largo,25.0,pollo
9,Terrier,corto,6.0,pavo


In [ ]:
# Vamos a calcular el porcentaje de nulos de la columna 'Peso_medio', mediana, y media.

round(df_perros.isna().sum()/df_perros.shape[0]*100, 2) # Tenemos un 20% de nulos

Raza           0.0
Pelo          10.0
Peso_medio    20.0
Comida_fav    10.0
dtype: float64

In [ ]:
df_perros['Peso_medio'].agg(['mean','median']) # La mediana es más representativa que la media

mean      14.4375
median    15.2500
Name: Peso_medio, dtype: float64

In [ ]:
# Nuestros nulos son el peso del dálmata y del bobtail. No tenemos otra columna para hacer un KNNImputer, por lo que hacemos un SimpleImputer

imputer = SimpleImputer(strategy='median')  # Creamos nuestro modelo

df_perros['Peso_medio'] = imputer.fit_transform(df_perros[['Peso_medio']]) # Se aplica nuestro modelo

In [ ]:
# Comida fav:10% Nulos, y Pelo igual podemos ver la moda, al ser una varibale categórica, y ver si es representativa.

df_perros['Comida_fav'].value_counts()
print("__________________")

df_perros['Pelo'].value_counts()

#Moda representativa "Pelo" --> corto
 # La moda es pollo, en Comida_fav --> df_perros['Comida_fav'].mode()

Comida_fav
pollo     4
salmon    2
beef      2
pavo      1
Name: count, dtype: int64

In [ ]:
#Bucle for, para columnas categóricas

for c in df_perros.columns:
    
    try:
        imputer = SimpleImputer(strategy='most_frequent')  # Creamos nuestro modelo

        df_perros[c] = imputer.fit_transform(df_perros[[c]]).ravel() # .ravel() lo convierte en 1D ((n,)), haciendo que pandas lo acepte al asignarlo a una columna
    except:
        c

In [ ]:
#En este caso, la sustitución de nulos es coherente. Pero, conociendo el campo de los datos, si hubieramos obtenido:
# que el doberman tiene el pelo largo, o que el Bobtail tiene una media de peso de 5 kg...tendríamos que solucionar
#a través de otro criterio.


In [12]:
display(df_perros)

,Raza,Pelo,Peso_medio,Comida_fav
0,Chihuahua,corto,3.00,salmon
1,Galgo,corto,17.00,pollo
2,Dalmata,corto,15.25,pollo
3,Boxer,corto,15.50,pollo
4,Braco,corto,15.00,beef
5,Doberman,corto,30.00,beef
6,Pequines,largo,4.00,salmon
7,Bobtail,largo,15.25,pollo
8,Pastor,largo,25.00,pollo
9,Terrier,corto,6.00,pavo
